In [1]:
import mlflow
import os
import pickle

In [2]:

mlflow.set_tracking_uri('http://127.0.0.1:5000')
mlflow.set_experiment("nyc-taxi-regression-assignment")

<Experiment: artifact_location='/Users/hsuyab/Documents/programming/work_related/mlops_zoomcap/week2_new/artifact/1', experiment_id='1', lifecycle_stage='active', name='nyc-taxi-regression-assignment', tags={}>

## Question1: mlflow version

In [3]:
print('mlflow verion:', mlflow.__version__)

mlflow verion: 1.26.1


## Question2: Download and preprocess the data

#### Preprocessing the data

In [4]:
!python preprocess_data.py --raw_data_path ./data --dest_path ./output


In [5]:
!ls -l ./output

total 18328
-rw-r--r--  1 hsuyab  staff   305256 Jun  2 23:14 dv.pkl
-rw-r--r--  1 hsuyab  staff  2805197 Jun  2 23:14 test.pkl
-rw-r--r--  1 hsuyab  staff  2661149 Jun  2 23:14 train.pkl
-rw-r--r--  1 hsuyab  staff  2166527 Jun  2 23:14 valid.pkl


In [6]:
print('Number of files saved in output folder:', len(os.listdir('./output')))

Number of files saved in output folder: 4


### Training a model with autolog

In [33]:
!python train.py

2022/06/02 23:31:58 INFO mlflow.tracking.fluent: Experiment with name 'nyc-taxi-regression-assignment' does not exist. Creating a new experiment.
2022/06/02 23:32:12 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "/Users/hsuyab/miniconda3/envs/mlops/lib/python3.8/site-packages/_distutils_hack/__init__.py:30: UserWarning: Setuptools is replacing distutils."


### Run hpo

In [8]:
!python hpo.py

100%|█████████| 50/50 [05:02<00:00,  6.06s/trial, best loss: 6.6284257482044735]


In [14]:
from mlflow.tracking import MlflowClient
client = MlflowClient(tracking_uri='http://127.0.0.1:5000')

In [15]:
runs = client.search_runs(experiment_ids=1, max_results=5, order_by=['metrics.rmse ASC'])

In [17]:
#runs

In [18]:
for run in runs:
    print(f"run id: {run.info.run_uuid}, rmse: {run.data.metrics['rmse']:.4f}")

run id: 87c4560f6d944f0a83db6f2d40e55790, rmse: 6.6284
run id: 1abc983c57f946f8985d45da3bd9947e, rmse: 6.6297
run id: 33dcf2c75d3f4aac9aff2cd50e2e9ccd, rmse: 6.6299
run id: 65c392e5959e4f829630d6b5f2230511, rmse: 6.6299
run id: 300deb7ea310459282d56a9abe100fc8, rmse: 6.6299


### Run the final register_model.py

In [21]:
!python register_model.py

2022/06/02 23:27:43 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "/Users/hsuyab/miniconda3/envs/mlops/lib/python3.8/site-packages/_distutils_hack/__init__.py:30: UserWarning: Setuptools is replacing distutils."
Successfully registered model 'random-forest-best-model'.
2022/06/02 23:28:40 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation.                     Model name: random-forest-best-model, version 1
Created version '1' of model 'random-forest-best-model'.


In [26]:
runs = client.search_runs(experiment_ids=2, max_results=5, order_by=['metrics.test_rmse ASC'])

In [27]:
for run in runs:
    print(f"run id: {run.info.run_uuid}, rmse: {run.data.metrics['test_rmse']:.4f}")

run id: e598924ce4584d128728b590fd0fa2cc, rmse: 6.5479
run id: f84460d2b23f45c1866a85f269febb72, rmse: 6.5489
run id: 8db222bcf5754db78faa3ee708bc94a7, rmse: 6.5491
run id: fef1ec2b3f954477bc42275f576cd3ad, rmse: 6.5491
run id: e21813776e784119a58a49fdff88325b, rmse: 6.5498


In [32]:
print(f"test rmse is: {runs[0].data.metrics['test_rmse']:.2f}")

test rmse is: 6.55
